In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [3]:
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [23]:
from keras.datasets import cifar10
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
Xnew = X_test[:4]/ 255.0

In [8]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [9]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [10]:
# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
# Compile model
epochs = 4
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

In [12]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 245s 5ms/step - loss: 1.7045 - acc: 0.3849 - val_loss: 1.3625 - val_acc: 0.5081
Epoch 2/25
50000/50000 [==============================] - 242s 5ms/step - loss: 1.3231 - acc: 0.5237 - val_loss: 1.1914 - val_acc: 0.5884
Epoch 3/25
50000/50000 [==============================] - 243s 5ms/step - loss: 1.1472 - acc: 0.5916 - val_loss: 1.0829 - val_acc: 0.6165
Epoch 4/25
50000/50000 [==============================] - 242s 5ms/step - loss: 1.0165 - acc: 0.6391 - val_loss: 1.0073 - val_acc: 0.6477
Epoch 5/25
50000/50000 [==============================] - 242s 5ms/step - loss: 0.9137 - acc: 0.6772 - val_loss: 0.9814 - val_acc: 0.6560
Epoch 6/25
 1760/50000 [>.............................] - ETA: 3:47 - loss: 0.8131 - acc: 0.7080

KeyboardInterrupt: 

In [13]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 66.30%


In [18]:
import keras
tbCallBack= keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq = 0,write_graph=True, write_images=True) 
#model.fit(...inputs andparameters..., callbacks=[tbCallBack])
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32, callbacks=[tbCallBack])

Train on 50000 samples, validate on 10000 samples
Epoch 1/4
50000/50000 [==============================] - 252s 5ms/step - loss: 0.8839 - acc: 0.6858 - val_loss: 0.9301 - val_acc: 0.6769
Epoch 2/4
50000/50000 [==============================] - 255s 5ms/step - loss: 0.6979 - acc: 0.7554 - val_loss: 0.8909 - val_acc: 0.6892
Epoch 3/4
50000/50000 [==============================] - 262s 5ms/step - loss: 0.6235 - acc: 0.7829 - val_loss: 0.8952 - val_acc: 0.6919
Epoch 4/4
50000/50000 [==============================] - 257s 5ms/step - loss: 0.5765 - acc: 0.7983 - val_loss: 0.8941 - val_acc: 0.6981


In [24]:
# new instances where we do not know the answer

#Xnew = scalar.transform(Xnew)
# make a prediction
ynew = model.predict_proba(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
    print("X=%s, Predicted=%s" % (Xnew[i], ynew[i]))

X=[[[0.61960784 0.62352941 0.64705882 ... 0.5372549  0.49411765 0.45490196]
  [0.59607843 0.59215686 0.62352941 ... 0.53333333 0.49019608 0.46666667]
  [0.59215686 0.59215686 0.61960784 ... 0.54509804 0.50980392 0.47058824]
  ...
  [0.26666667 0.16470588 0.12156863 ... 0.14901961 0.05098039 0.15686275]
  [0.23921569 0.19215686 0.1372549  ... 0.10196078 0.11372549 0.07843137]
  [0.21176471 0.21960784 0.17647059 ... 0.09411765 0.13333333 0.08235294]]

 [[0.43921569 0.43529412 0.45490196 ... 0.37254902 0.35686275 0.33333333]
  [0.43921569 0.43137255 0.44705882 ... 0.37254902 0.35686275 0.34509804]
  [0.43137255 0.42745098 0.43529412 ... 0.38431373 0.37254902 0.34901961]
  ...
  [0.48627451 0.39215686 0.34509804 ... 0.38039216 0.25098039 0.33333333]
  [0.45490196 0.4        0.33333333 ... 0.32156863 0.32156863 0.25098039]
  [0.41960784 0.41176471 0.34901961 ... 0.30196078 0.32941176 0.2627451 ]]

 [[0.19215686 0.18431373 0.2        ... 0.14117647 0.14117647 0.12941176]
  [0.2        0.1568